In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


Load the wikipedia page using the `requests` module and convert it to lxml using `BeautifulSoup`

In [2]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
contents = page.content
soup = BeautifulSoup(contents, 'lxml') #converter para html
#print(soup.prettify())

Retrieve the wikipedia table, which is of the class *wikitable sortable* and extract the relevant information

In [3]:
My_table = soup.find('table',{'class':'wikitable sortable'})
links = My_table.findAll('td')
nl = str(links).replace('[','').replace(']','').split('\n</td>, ')

#for link in nl: print(link)

postalcodes = []
boroughs = []
neighborhoods = []

for i in range(len(nl)):
        postalcode = nl[i].split('>')[1].split('<')[0]
        postalcodes.append(postalcode)
        
        if 'title' in nl[i]:
            borough = nl[i].split('title="')[1].split('"')[0]
            boroughs.append(borough.replace(', Toronto', '').replace(' (Toronto)', '')) # remove unnecessary information
        else:
            borough = nl[i].split(' <td>')[1].split('<')[0]
            boroughs.append(borough)
            
        string = nl[i].strip('</a>').split('>')[-1] #the neighborhood is the last substring
        neighborhood = string if string != 'Not assigned' else borough # if the neighborhood is not assigned, then it should be assigned the borough's name
        neighborhoods.append(neighborhood)   

Keep only the unique values from the PostalCode and Borough columns and join the unique values from Neighborhood column and assign them to a pandas dataframe

In [4]:
df = pd.DataFrame({'PostalCode':postalcodes, 'Borough': boroughs, 'Neighborhood':neighborhoods})
df = df[df.Borough != 'Not assigned'] #remove rows where the Borough is not assigned
# keep only the unique values from the Postal Co

df_PostalCodes = df.groupby(['PostalCode','Borough']).Neighborhood.unique().apply(lambda x: "%s" % ', '.join(x)).to_frame().reset_index().sort_values('PostalCode')

#canadaPostalCodes[canadaPostalCodes.PostalCode == 'M5G']#.values
df_PostalCodes.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Question2: Obtain the Geographical Coordinates for each one of the Postal Codes

I tried running the code below, but it keeps running "forever", so I converted it to markdown for showcasing purposes.

Hence, i downloaded the given csv file and merged the two new columns on coincident rows, thus obtaining the latitude and longitude for each one of the Postal Codes

In [5]:
long_lat_df = pd.read_csv('http://cocl.us/Geospatial_data') #download the Geospatial Coordinates
long_lat_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the `df_PostalCodes` and `long_lat_df` data frames into one names `canadaPostalCodes`

In [6]:
canadaPostalCodes = pd.concat([df_PostalCodes, long_lat_df.iloc[:,1:]], axis=1, sort=False)
canadaPostalCodes.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
